<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/final_bot_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 2s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [2]:
!pip install flask flask-ngrok tensorflow joblib pykalman scipy

from flask import Flask, jsonify
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from scipy.signal import butter, filtfilt
from pykalman import KalmanFilter
import subprocess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 13.2 MB/s eta 0:00:00


In [8]:
app = Flask(__name__)

# Setup for your EEG signal processing
sampling_rate_original = 2500
sampling_rate_final = 200
freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

def bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

def downsample(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]

def generate_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg = []
    for _ in range(62):
        alpha = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, 3, t.shape)
        signal = delta + theta + alpha + beta + gamma + noise
        eeg.append(np.clip(signal, -70, 70))
    return np.array(eeg)

def extract_de_lds():
    eeg_data = generate_eeg()
    de_feats = []
    for ch in eeg_data:
        ch = downsample(ch, sampling_rate_original, sampling_rate_final)
        band_feats = []
        for _, (low, high) in freq_bands.items():
            filtered = bandpass_filter(ch, low, high, sampling_rate_final)
            var = np.var(filtered)
            de = 0.5 * np.log(2 * np.pi * np.e * var + 1e-8)
            band_feats.append(de)
        smoothed, _ = kf.filter(np.array(band_feats).reshape(-1, 1))
        de_feats.append(smoothed.flatten())
    de_feats = np.array(de_feats)
    de_min, de_max = np.min(de_feats), np.max(de_feats)
    scaled = 15 + (de_feats - de_min) / (de_max - de_min + 1e-8) * (27 - 15)
    return scaled.flatten()

@app.route('/predict', methods=['GET'])
def predict():
    sample = extract_de_lds().reshape((1, 1, 310))
    model = load_model('eeg_emotion_bilstm_model(100e).h5')
    encoder = joblib.load('eeg_label_encoder.pkl')
    prediction = model.predict(sample)
    # print(f"Model Prediction: {prediction}")

    label = encoder.inverse_transform(np.argmax(prediction, axis=1))[0]
    print(f"Predicted Emotion: {label}")
    return jsonify({'emotion': label})

# Run LocalTunnel to expose the Flask API
# Run LocalTunnel to expose the Flask API with a custom subdomain for static URL
def run_localtunnel():
    subdomain = "eegemotion"  # Replace with your desired subdomain
    process = subprocess.Popen(['lt', '--port', '5000', '-s', subdomain], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode('utf-8').strip())  # Show the URL output from LocalTunnel

# Start the LocalTunnel process and Flask app
if __name__ == "__main__":
    from threading import Thread
    # Run LocalTunnel in a separate thread
    thread = Thread(target=run_localtunnel)
    thread.start()
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://eegemotion.loca.lt


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step


INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:25] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Fear


In [ ]:
neutral, disgust, anger, surprise, happy